# Healthy Streets of Los Angeles Injuries/Deaths vs implemented Mobility in plan 2022
This project compares a number of car accidents in 2022 resulted in deaths/injuries for streets with and without Mobility Plan safety improvements.

Sources:
* Injuries/Deaths in the City of LA https://tims.berkeley.edu/
* Implepemented Mobility Plan shapefiles (see hsla_mobilty_plan project)

Assumptions:
* Location only City of Los Angeles
* year is 2022
* Excluding freeways (STATE_ROUTE is Null)

Output:
* deaths/injuries on LA streets last year happened on streets missing mobility safety plan improvements

Any questions - elena.sunchugasheva@gmail.com

In [2]:
import pandas as pd
import datetime
import geopandas as gpd
from shapely.geometry import Polygon, LineString, Point
import folium

pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 1000)

## functions

In [130]:
def show_map(gdf_names):
    '''
    show a number of gdfs with set colors
    - gdfs_colors - dictionary with format:{'name': gdf}
    ''' 
    colors = ['green', 'blue', 'red', 'orange', 'purple', 'yellow', 'magenta']
    f = folium.Figure(width=1000, height=500)
    m = folium.Map(location = [34.05, -118.24], zoom_start=10).add_to(f)
    i = 0
    
    for name in gdf_names.keys():
        color = colors[i]
        i += 1
        gdf = gdf_names[name]
        print(f'{color}: {name}')
        if gdf.loc[0, 'geometry'].geom_type!='Point':
            gdf.explore(
                m = m,
                name = name,
                color = color
            )
        else:
            #folium.Marker(gdf).add_to(m)
            folium.features.GeoJson(gdf).add_to(m)

    folium.TileLayer(
        'CartoDB positron',
        show=False
    ).add_to(m) 
    folium.LayerControl().add_to(m)
    
    return m

In [12]:
def buffer(gdf, radius, proj='EPSG:4326', proj_calc='EPSG:3857'):
    '''
    convert a gdf of linestrings into a gdf of polygons with radius
    - gdf - GeoDataFrame, has column "geometry"
    - radius - radius of bufferm meters
    - proj - projection of the original dataset
    - proj_calc='EPSG:3857' - projection for calculations
    '''  
    gdf = gdf.copy()
    gdf_calc = gdf.to_crs(proj_calc)
    #print('data proj:', proj, '\ncalculation proj: ', proj_calc)
    gdf['buffered'] = gdf_calc.buffer(radius, cap_style=2).to_crs(proj)
    gdf.set_geometry('buffered', inplace=True)
    
    # merge all intersecting buffered polygons
    gdf_return = gpd.GeoDataFrame(
        geometry=[gdf.unary_union]
    ).explode(
        index_parts=False
    ).reset_index(
        drop=True
    )
    gdf_return.geometry.crs = proj
    
    return gdf_return

# data

In [148]:
today = datetime.datetime.now()
print(today)
radius = 20 # our average street will be 40m wide

2023-12-08 02:55:00.497280


## data preparation

### get crashes data

In [4]:
crashes_raw = pd.read_csv('Crashes.csv')
display(crashes_raw.head(1))

,CASE_ID,ACCIDENT_YEAR,PROC_DATE,JURIS,COLLISION_DATE,COLLISION_TIME,OFFICER_ID,REPORTING_DISTRICT,DAY_OF_WEEK,CHP_SHIFT,POPULATION,CNTY_CITY_LOC,SPECIAL_COND,BEAT_TYPE,CHP_BEAT_TYPE,CITY_DIVISION_LAPD,CHP_BEAT_CLASS,BEAT_NUMBER,PRIMARY_RD,SECONDARY_RD,DISTANCE,DIRECTION,INTERSECTION,WEATHER_1,WEATHER_2,STATE_HWY_IND,CALTRANS_COUNTY,CALTRANS_DISTRICT,STATE_ROUTE,ROUTE_SUFFIX,POSTMILE_PREFIX,POSTMILE,LOCATION_TYPE,RAMP_INTERSECTION,SIDE_OF_HWY,TOW_AWAY,COLLISION_SEVERITY,NUMBER_KILLED,NUMBER_INJURED,PARTY_COUNT,PRIMARY_COLL_FACTOR,PCF_CODE_OF_VIOL,PCF_VIOL_CATEGORY,PCF_VIOLATION,PCF_VIOL_SUBSECTION,HIT_AND_RUN,TYPE_OF_COLLISION,MVIW,PED_ACTION,ROAD_SURFACE,ROAD_COND_1,ROAD_COND_2,LIGHTING,CONTROL_DEVICE,CHP_ROAD_TYPE,PEDESTRIAN_ACCIDENT,BICYCLE_ACCIDENT,MOTORCYCLE_ACCIDENT,TRUCK_ACCIDENT,NOT_PRIVATE_PROPERTY,ALCOHOL_INVOLVED,STWD_VEHTYPE_AT_FAULT,CHP_VEHTYPE_AT_FAULT,COUNT_SEVERE_INJ,COUNT_VISIBLE_INJ,COUNT_COMPLAINT_PAIN,COUNT_PED_KILLED,COUNT_PED_INJURED,COUNT_BICYCLIST_KILLED,COUNT_BICYCLIST_INJURED,COUNT_MC_KILLED,COUNT_MC_INJURED,PRIMARY_RAMP,SECONDARY_RAMP,LATITUDE,LONGITUDE,COUNTY,CITY,POINT_X,POINT_Y
0,8040740,2016,2016-05-24,1942,2016-05-05,1310,38356,1779,4,5,7,1942,0,0,0,Q,0,9TL57,NORDHOFF ST,HASKELL AV,0.0,NaN,Y,B,-,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,3,0,1,2,A,-,12,21453.0,A,N,D,C,A,A,H,-,A,A,0,NaN,NaN,NaN,NaN,Y,NaN,-,-,0,1,0,0,0,0,0,0,0,-,-,NaN,NaN,LOS ANGELES,LOS ANGELES,-118.47634,34.23557


In [61]:
# columns we are interested in
crashes_col = [
    'CASE_ID', 'COUNTY', 'CITY',
    'ACCIDENT_YEAR', 
    'PRIMARY_RD', 'SECONDARY_RD', 'POINT_X', 'POINT_Y',
    'INTERSECTION', 'DISTANCE', 
    'NUMBER_KILLED', 'NUMBER_INJURED', 
    'COUNT_PED_KILLED', 'COUNT_PED_INJURED',
    'COUNT_BICYCLIST_KILLED', 'COUNT_BICYCLIST_INJURED'
]

# take only City of LA and not highways
crashes = crashes_raw[
        (crashes_raw.CITY=='LOS ANGELES')&
        crashes_raw.STATE_ROUTE.isnull()&
        (crashes_raw.ACCIDENT_YEAR==2022)
    ][crashes_col].copy().reset_index(drop=True)
display(crashes.head(1))

,CASE_ID,COUNTY,CITY,ACCIDENT_YEAR,PRIMARY_RD,SECONDARY_RD,POINT_X,POINT_Y,INTERSECTION,DISTANCE,NUMBER_KILLED,NUMBER_INJURED,COUNT_PED_KILLED,COUNT_PED_INJURED,COUNT_BICYCLIST_KILLED,COUNT_BICYCLIST_INJURED
0,9552804,LOS ANGELES,LOS ANGELES,2022,MONTEREY RD,CASSATT ST,-118.188545,34.089386,N,150.0,0,2,0,0,0,0


In [139]:
#create geodataframe
crashes_gdf = gpd.GeoDataFrame(
    crashes[['CASE_ID', 'NUMBER_KILLED', 'NUMBER_INJURED']],
    geometry=gpd.points_from_xy(crashes['POINT_X'], crashes['POINT_Y'])
)
crashes_gdf.crs = 'EPSG:4326'

take a look at stats/outliers

In [7]:
print(
    'intersections:', crashes[crashes.INTERSECTION=='Y'].shape[0],
    ', non-intersections:', crashes[crashes.INTERSECTION=='N'].shape[0],
    ', total:', crashes.shape[0]
)

intersections: 2475 , non-intersections: 3877 , total: 6381


In [8]:
crashes[[
    'NUMBER_KILLED', 'NUMBER_INJURED',
    'COUNT_PED_KILLED', 'COUNT_PED_INJURED',
    'COUNT_BICYCLIST_KILLED', 'COUNT_BICYCLIST_INJURED'
]].describe()

,NUMBER_KILLED,NUMBER_INJURED,COUNT_PED_KILLED,COUNT_PED_INJURED,COUNT_BICYCLIST_KILLED,COUNT_BICYCLIST_INJURED
count,6381.000000,6381.000000,6381.000000,6381.000000,6381.000000,6381.000000
mean,0.042940,1.394922,0.019746,0.192290,0.002351,0.067074
std,0.211812,0.925162,0.141373,0.446347,0.057324,0.253901
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000
max,3.000000,16.000000,2.000000,8.000000,3.000000,3.000000


### get mobility_implemented data

In [138]:
files_implemented = [
    'unprotected_bike_lane_implemented_2023_11_29',
    'protected_bike_lane_implemented_2023_11_29',
    'NEN_bike_lane_implemented_2023_11_29',
    'class1_bike_lane_implemented_2023_12_06',
    'protected_bus_lane_implemented_2023_11_29'
]

mobility_implemented = gpd.GeoDataFrame()
for file in files_implemented:
    geo_file = open(f'../hsla_mobility_plan/{file}.geojson')
    gdf = gpd.read_file(geo_file)
    gdf['name'] = file[:-11]
    display(gdf.head(1))
    if mobility_implemented.shape[0]==0:
        mobility_implemented = gdf[['name', 'length_m', 'geometry']].copy()
    else:
        mobility_implemented = pd.concat([
            mobility_implemented,
            gdf[['name', 'length_m', 'geometry']]]
        )

,OBJECTID,BICYCLE_N,length_m,geometry,name
0,1,2,450.88124,"LINESTRING (-118.35923 34.22239, -118.36181 34...",unprotected_bike_lane_implemented


,OBJECTID,BICYCLE_N,length_m,geometry,name
0,206,1,147.987411,"LINESTRING (-118.53610 34.23077, -118.53610 34...",protected_bike_lane_implemented


,OBJECTID,CF,CASE_NUM,SOURCE,ADOPTDATE,CPA_1,CPA_2,NEIGHBORHD_N,Shape__Length,length_m,geometry,name
0,14,15-0719,CPC-2013-910-GPA-SP-CA-MSC-M2,Hollywood Community Plan (6/19/12),2016-09-09 00:00:00+00:00,HWD,None,2,36.727754,36.727754,"LINESTRING (-118.29289 34.09551, -118.29321 34...",NEN_bike_lane_implemented


,OBJECTID,length_m,geometry,name
0,11,4776.592991,"LINESTRING (-118.25434 34.10786, -118.25285 34...",class1_bike_lane_implemented


,OBJECTID,CF,CASE_NUM,SOURCE,ADOPTDATE,CPA_1,CPA_2,TRANSIT_N,Shape__Length,length_m,geometry,name
0,9,15-0719,CPC-2013-910-GPA-SP-CA-MSC,Downtown Street Standards,2015-08-11 00:00:00+00:00,CCY,None,3,134.114935,134.114935,"LINESTRING (-118.24713 34.04347, -118.24617 34...",protected_bus_lane_implemented


In [147]:
mobility_implemented_buffer = buffer(mobility_implemented, radius = radius)

### get mobility_unimplemented data

In [140]:
files_plan = [
    'Bicycle_Enhanced_Network_Paths',
    'Bicycle_Enhanced_Network',
    'Neighborhood_Enhanced_Network',
    'Transit_Enhanced_Network'
]

mobility_plan = gpd.GeoDataFrame()
for file in files_plan:
    geo_file = open(f'../hsla_mobility_plan/{file}.geojson')
    gdf = gpd.read_file(geo_file)
    gdf['name'] = file
    display(gdf.head(1))
    if mobility_plan.shape[0]==0:
        mobility_plan = gdf[['name', 'Shape__Length', 'geometry']].copy()
    else:
        mobility_plan = pd.concat([
            mobility_plan,
            gdf[['name', 'Shape__Length', 'geometry']]]
        )

,OBJECTID_12,OBJECTID_1,OBJECTID,bikewaytyp,exbikeway,BIKEWYNAME,MILEAGE,COUNDIST,NETWORK,ARTERIAL,Shape_Leng,Shape_Le_1,Shape__Length,geometry,name
0,1,1,1,Planned BP,0,Valley LA River Path,0.32488,4,GREEN,0,1715.368583,1715.368583,631.219551,"LINESTRING (-118.35051 34.14248, -118.34966 34...",Bicycle_Enhanced_Network_Paths


,OBJECTID,CF,CASE_NUM,SOURCE,ADOPTDATE,COMMENTS,CPA_1,CPA_2,BICYCLE_N,created_user,created_date,last_edited_user,last_edited_date,Shape__Length,geometry,name
0,1,15-0719,CPC-2013-910-GPA-SP-CA-MSC,,2015-08-11 00:00:00+00:00,Updated per Timmy. Middle out consistency edit.,SVY,None,2,None,1970-01-01 00:00:00+00:00,None,1970-01-01 00:00:00+00:00,450.88124,"LINESTRING (-118.35923 34.22239, -118.36181 34...",Bicycle_Enhanced_Network


,OBJECTID,CF,CASE_NUM,SOURCE,ADOPTDATE,CPA_1,CPA_2,NEIGHBORHD_N,Shape__Length,geometry,name
0,1,15-0719,CPC-2013-910-GPA-SP-CA-MSC-M2,,2016-09-09 00:00:00+00:00,SLK,None,2,139.333326,"LINESTRING (-118.25885 34.08532, -118.25835 34...",Neighborhood_Enhanced_Network


,OBJECTID,CF,CASE_NUM,SOURCE,ADOPTDATE,CPA_1,CPA_2,TRANSIT_N,Shape__Length,geometry,name
0,1,15-0719,CPC-2013-910-GPA-SP-CA-MSC,,2015-08-11 00:00:00+00:00,WCH,None,1,428.828885,"LINESTRING (-118.37112 33.94526, -118.37497 33...",Transit_Enhanced_Network


In [146]:
 # intersect polygons of actual and planned paths
mobility_unimplemented = gpd.overlay(
    mobility_plan, 
    implemented_buffer, 
    how='difference',
    keep_geom_type=False
)
display(mobility_unimplemented.head())

,name,Shape__Length,geometry
0,Bicycle_Enhanced_Network_Paths,631.219551,"LINESTRING (-118.35051 34.14248, -118.34966 34..."
1,Bicycle_Enhanced_Network_Paths,2632.455613,"LINESTRING (-118.22734 34.08124, -118.22520 34..."
2,Bicycle_Enhanced_Network_Paths,694.062760,"LINESTRING (-118.23389 34.09827, -118.23886 34..."
3,Bicycle_Enhanced_Network_Paths,3009.990132,"LINESTRING (-118.22564 34.07965, -118.22455 34..."
4,Bicycle_Enhanced_Network_Paths,4748.604236,"MULTILINESTRING ((-118.34502 34.14238, -118.34..."


In [144]:
mobility_plan.shape[0], mobility_unimplemented.shape[0]

(24936, 22611)

In [149]:
mobility_unimplemented_buffer = buffer(mobility_unimplemented, radius = radius)

## calculation

In [154]:
mobility_implemented_crashes = gpd.GeoDataFrame()
for idx, row in mobility_implemented_buffer.iterrows():
    intersection = crashes_gdf[crashes_gdf.covered_by(row['geometry'])]
    mobility_implemented_crashes = pd.concat([
            mobility_implemented_crashes,
            intersection
        ])
print(mobility_implemented_crashes.shape[0])

1125


In [152]:
mobility_unimplemented_crashes = gpd.GeoDataFrame()
for idx, row in mobility_unimplemented_buffer.iterrows():
    intersection = crashes_gdf[crashes_gdf.covered_by(row['geometry'])]
    mobility_unimplemented_crashes = pd.concat([
            mobility_unimplemented_crashes,
            intersection
        ])
print(mobility_unimplemented_crashes.shape[0])

In [155]:
mobility_unimplemented_crashes.head()

,CASE_ID,NUMBER_KILLED,NUMBER_INJURED,geometry
2,9442714,0,1,POINT (-118.48518 34.22501)
4,9561533,0,1,POINT (-118.35287 34.09802)
5,9411571,0,1,POINT (-118.50883 34.16070)
14,9403658,0,1,POINT (-118.43117 34.20938)
20,9449251,0,5,POINT (-118.53604 34.19385)


In [165]:
# show_map({
#     'implemented': mobility_implemented_buffer,
#     'crashes': mobility_implemented_crashes
# })

In [164]:
print(
    f'Number of crashes in LA with an injury/death in 2022 for streets where\
    \nMobility plan was implented:\
    {mobility_implemented_crashes.shape[0]},\
    \nMobility plan not implented:\
    {mobility_unimplemented_crashes.shape[0]},\
    \ntotal crashes regardless of MP:\
    {crashes_gdf.shape[0]}'
)

Number of crashes in LA with an injury/death in 2022 for streets where    
Mobility plan was implented:    1125,    
Mobility plan not implented:    3591,    
total crashes regardless of MP:    6381
